In [1]:
# Display active instances
spark

In [2]:
path = "/data/students/bigdata_internet/dataframe_examples/mycsv.csv"
df = spark.read.load(path, format="csv", header=True, inferSchema=True)

In [3]:
df.collect()

[Row(name='Paolo', age=50, dept='DET'),
 Row(name='Luca', age=40, dept='DAUIN'),
 Row(name='Martino', age=30, dept='DAD'),
 Row(name='Paolo', age=25, dept='DIST'),
 Row(name='Francesca', age=40, dept='DAUIN'),
 Row(name='Paolo', age=32, dept='DET'),
 Row(name='Luca', age=56, dept='DAD')]

In [4]:
df.show()

+---------+---+-----+
|     name|age| dept|
+---------+---+-----+
|    Paolo| 50|  DET|
|     Luca| 40|DAUIN|
|  Martino| 30|  DAD|
|    Paolo| 25| DIST|
|Francesca| 40|DAUIN|
|    Paolo| 32|  DET|
|     Luca| 56|  DAD|
+---------+---+-----+



In [5]:
df.groupBy("name").show()

AttributeError: 'GroupedData' object has no attribute 'show'

In [8]:
# Group by name and report average age and count how many elements have the same name
df_group = df.groupBy("name").agg({"age":"avg", "name":"count"})
df_group.show()

+---------+-----------+------------------+
|     name|count(name)|          avg(age)|
+---------+-----------+------------------+
|Francesca|          1|              40.0|
|    Paolo|          3|35.666666666666664|
|     Luca|          2|              48.0|
|  Martino|          1|              30.0|
+---------+-----------+------------------+



In [9]:
# Sort in ascending order depending name (alphabetical) and, if same, on age
sorted_df = df.sort("name", "age", ascending=True)
sorted_df.show()

+---------+---+-----+
|     name|age| dept|
+---------+---+-----+
|Francesca| 40|DAUIN|
|     Luca| 40|DAUIN|
|     Luca| 56|  DAD|
|  Martino| 30|  DAD|
|    Paolo| 25| DIST|
|    Paolo| 32|  DET|
|    Paolo| 50|  DET|
+---------+---+-----+



In [24]:
# Join the two DFs according to the same name
joined_df = df_group.join(sorted_df, 'name')
joined_df.show()

+---------+-----------+------------------+---+-----+
|     name|count(name)|          avg(age)|age| dept|
+---------+-----------+------------------+---+-----+
|Francesca|          1|              40.0| 40|DAUIN|
|     Luca|          2|              48.0| 40|DAUIN|
|     Luca|          2|              48.0| 56|  DAD|
|  Martino|          1|              30.0| 30|  DAD|
|    Paolo|          3|35.666666666666664| 25| DIST|
|    Paolo|          3|35.666666666666664| 32|  DET|
|    Paolo|          3|35.666666666666664| 50|  DET|
+---------+-----------+------------------+---+-----+



In [27]:
joined_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- count(name): long (nullable = false)
 |-- avg(age): double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dept: string (nullable = true)



In [29]:
joined_df.selectExpr("name", "count(name) AS count", "avg(age) AS avg", "age", "dept").show()

AnalysisException: "grouping expressions sequence is empty, and '`name`' is not an aggregate function. Wrap '(count(`name`) AS `count`, avg(CAST(`age` AS BIGINT)) AS `avg`)' in windowing function(s) or wrap '`name`' in first() (or first_value) if you don't care which value you get.;;\nAggregate [name#10, count(name#10) AS count#509L, avg(cast(age#473 as bigint)) AS avg#510, age#473, dept#474]\n+- Project [name#10, count(name)#105L, avg(age)#106, age#473, dept#474]\n   +- Join Inner, (name#10 = name#472)\n      :- Aggregate [name#10], [name#10, count(name#10) AS count(name)#105L, avg(cast(age#11 as bigint)) AS avg(age)#106]\n      :  +- Relation[name#10,age#11,dept#12] csv\n      +- Sort [name#472 ASC NULLS FIRST, age#473 ASC NULLS FIRST], true\n         +- Relation[name#472,age#473,dept#474] csv\n"